In [9]:
using DataStructures
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using BenchmarkTools

## Funzione CIRCLE()

In [2]:
function circle(radius=1., angle=2*pi)
    function circle0(shape=[36])
        V, EV = cuboidGrid(shape)
        V = (angle/shape[1])*V
        V = hcat(map(u->[radius*cos(u); radius*sin(u)], V)...)
        W, EW = simplifyCells(V, EV)
        return W, EW
    end
    return circle0
end

circle (generic function with 3 methods)

In [3]:
@btime W,CW = Lar.circle()();

  116.760 μs (1838 allocations: 139.64 KiB)


In [4]:
@code_warntype circle()();

Variables
  #self#::var"#circle0#2"{Float64,Float64}

Body::Tuple{Any,Any}
1 ─ %1 = Base.vect(36)::Array{Int64,1}
│   %2 = (#self#)(%1)::Tuple{Any,Any}
└──      return %2


---------------------------------------------------------------------------------------------------------------

La funzione CIRCLE è una funzione che ci consente di costruire un cerchio di raggio 1 in formato 2D , è una funzione che fa solamente quello . Uno degli scopi del progetto era suddividere funzioni che svolgono due o più compiti insieme in funzioni che svolgono uno ed un solo compito. In questo caso circle crea solamente un cerchio . Però per rendere più leggibile il codice spezzo la funzione circle , non ho riscontrato dei miglioramenti .... 

Per ridefinire CIRCLE ho bisogno delle funzioni di supporto ad essa , uso simplifyCellsRef ovvero la funzione principale a cui ho fatto refactoring .

In [2]:
function simplifyCellsRef(V,CV)
    PRECISION = 5
    vertDict = DefaultDict{Array{Float64,1}, Int64}(0)
    index = 0
    W = Array{Float64,1}[]
    FW = Array{Int64,1}[]
    
    for incell in CV
        outcell = Int64[]
        for v in incell
         vert = V[:,v]
         key = map(approxVal(PRECISION), vert)
            if vertDict[key]==0
                index += 1
                vertDict[key] = index
                push!(outcell, index)
                push!(W,key)
            else
                push!(outcell, vertDict[key])
            end
        end
         push!(FW, [Set(outcell)...])
    end
    return hcat(W...),FW
end

simplifyCellsRef (generic function with 1 method)

In [15]:
function approxVal(PRECISION)
    function approxVal0(value)
        out = round(value, digits=PRECISION)
        if out == -0.0
            out = 0.0
        end
        return out
    end
    return approxVal0
end

approxVal (generic function with 1 method)

In [29]:
function forCircle(radius,V)
  hcat(map(u->[radius*cos(u); radius*sin(u)], V)...)
end

forCircle (generic function with 3 methods)

In [35]:
 function circle2(radius=1., angle=2*pi)
    function circle0(shape=[36])
        V, EV = Lar.cuboidGrid(shape)
        V = (angle/shape[1])*V
        L = forCircle(radius,V)
        W, EW = simplifyCellsRef(L, EV)
        return W, EW
    end
    return circle0
end

@btime W,CW = circle2()();

  137.461 μs (2093 allocations: 144.20 KiB)


Potrei ridefinire una funzione che svolge lo stesso compito di map , ma studiando ho capito che map è molto piu veloce su dimensioni di array grandi oltre ad avere una sintattica piu elegante e coincisa . 

In [36]:
@code_warntype circle2()();

Variables
  #self#::var"#circle0#47"{Float64,Float64}

Body::Tuple{Array,Array{Array{Int64,1},1}}
1 ─ %1 = Base.vect(36)::Array{Int64,1}
│   %2 = (#self#)(%1)::Tuple{Array,Array{Array{Int64,1},1}}
└──      return %2


Con la macro @code_warntype noto delle differenze rispetto alla precedente ..